In [17]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import os
import random

from functions.parse_data import synth_dataloader
from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from cot_train.utils import MLP5

In [18]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [19]:
#Load data and inspect
df = synth_dataloader('SMHIdata2_newsurf')
df.head(10)

,Cloud_B02,Cloud_B03,Cloud_B04,Cloud_B05,Cloud_B06,Cloud_B07,Cloud_B08,Cloud_B08A,Cloud_B09,Cloud_B10,...,Clear_B11,Clear_B12,Sat_Zenith_Angle,Sun_Zenith_Angle,Azimuth_Diff_Angle,COT,Cloud_Type,Profile_ID,GOT,Water_Vapor
0,0.57804,0.51792,0.54680,0.56000,0.56824,0.57365,0.56583,0.57950,0.37962,0.01949,...,0.04745,0.03758,9.00,68.68,48.59,25.181,3,9543,0.122,0.56
1,0.28975,0.25479,0.29171,0.32868,0.40295,0.44817,0.45809,0.50999,0.17019,0.00067,...,0.75602,0.58746,1.60,73.05,176.23,1.730,2,3672,0.116,0.77
2,0.71170,0.68907,0.73376,0.76922,0.81416,0.83261,0.83383,0.85262,0.63399,0.10050,...,0.65577,0.52408,14.75,42.45,16.45,20.746,4,3564,0.124,0.23
3,0.30316,0.32260,0.39997,0.43564,0.48821,0.52882,0.53207,0.58302,0.22735,0.00072,...,0.87771,0.74346,7.49,55.96,96.60,0.721,1,2993,0.122,0.83
4,0.84968,0.80047,0.83504,0.80686,0.83849,0.86902,0.80830,0.88989,0.26912,0.00051,...,0.81451,0.56093,1.45,51.76,79.44,49.984,3,6226,0.127,4.57
5,0.47968,0.45684,0.48555,0.50933,0.63299,0.71180,0.65449,0.78567,0.27385,0.01380,...,0.91439,0.60915,9.17,64.44,87.17,11.128,2,5251,0.128,4.42
6,0.31475,0.34345,0.35773,0.44913,0.72468,0.78040,0.75561,0.80269,0.34052,0.00156,...,0.90172,0.66072,10.77,68.43,122.12,4.445,1,6703,0.080,0.71
7,0.71760,0.68823,0.71453,0.72291,0.81518,0.87488,0.82043,0.93069,0.37696,0.04403,...,0.94217,0.81028,10.29,41.28,9.38,22.209,5,3031,0.128,2.78
8,0.63913,0.63027,0.67796,0.71480,0.81141,0.86388,0.85365,0.89979,0.65188,0.33643,...,0.89019,0.62145,7.52,48.72,141.93,18.855,4,8434,0.101,0.58
9,0.24895,0.24306,0.24877,0.31625,0.51416,0.57959,0.58756,0.64406,0.27037,0.00287,...,0.75560,0.52477,13.00,67.88,100.70,0.664,5,2395,0.124,0.57


In [20]:
#Set columns for X and y (input/output features)
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Sun_Zenith_Angle']
y_cols = ['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
          'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

#Find X and y
X=df[X_cols]
y=df[y_cols]


#Separate testdata from rest for 80/10/10 Train/Val/Test split
X_trainval, X_test, y_trainval, y_test=train_test_split(X,y,test_size=0.1,random_state=313)

#Add noise to X_test, 0 mean with stdev equal to 3% of mean of each feature
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X.to_numpy(),axis=0)*0.03

In [21]:
#Set up which quantiles to estimate, and find index of estimator (q=0.5)
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()

#Set up algorithm parameters for both cases
val_size=0.1
num_models=5 #Set number of models in ensemble
batch_size=500
nepochs=1000
noise_ratio = 0.03
early_break=True
clear_noise = True

In [22]:
#Choose if to save models and data, if so set paths
save_load=True
if save_load:
    test_name_1 = "Tuned_Untuned-Untuned_newsurf"
    main_filepath_1 = 'pytorch_models/'+test_name_1
    test_name_2 = "Tuned_Untuned-Tuned_newsurf"
    main_filepath_2 = 'pytorch_models/'+test_name_2

Case 1: Before tuning

In [23]:
#Set specific parameters
lr=0.003
no_nodes=100

#Set up NW
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break,
            clear_noise=clear_noise)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_1+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

Epoch 431


Batch number: 100%|██████████| 320/320 [00:00<00:00, 362.36it/s]


Training loss [2.5139306] Validation loss [2.5375323]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 331
With validation loss: 2.5069756507873535


In [24]:
#Load models
if save_load:
    base_path = main_filepath_1 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
untuned_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','NMSE','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=np.linalg.norm(y_test.to_numpy()-preds[:,:,est],axis=(0,1))**2
    norm=np.linalg.norm(y_test.to_numpy(),axis=(0,1))**2
    nmse=mse/norm
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,nmse,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','NMSE','Mean_Quantile_Loss'])
    untuned_model_metrics=pd.concat([untuned_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=np.linalg.norm(y_test.to_numpy()-preds_total[:,:,est],axis=(0,1))**2
norm=np.linalg.norm(y_test.to_numpy(),axis=(0,1))**2
nmse=mse/norm
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,nmse,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','NMSE','Mean_Quantile_Loss'])
untuned_model_metrics=pd.concat([untuned_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    untuned_model_metrics=untuned_model_metrics.reset_index(drop=True)
    untuned_model_metrics.to_csv(main_filepath_1+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_124826/3046067018.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  untuned_model_metrics=pd.concat([untuned_model_metrics,tmp_metrics])


Case 2: After tuning

In [25]:
#Set specific parameters
lr=0.002
no_nodes=200

#Create network
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break,
            clear_noise=clear_noise)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_2+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

Epoch 421


Batch number: 100%|██████████| 320/320 [00:01<00:00, 313.10it/s]

Training loss [2.4674404] Validation loss [2.4907465]
Epoch 422



Batch number: 100%|██████████| 320/320 [00:01<00:00, 313.09it/s]

Training loss [2.470353] Validation loss [2.4688263]
Epoch 423



Batch number: 100%|██████████| 320/320 [00:01<00:00, 312.45it/s]

Training loss [2.4719076] Validation loss [2.4716926]
Epoch 424



Batch number: 100%|██████████| 320/320 [00:01<00:00, 312.42it/s]

Training loss [2.4634838] Validation loss [2.4692824]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 324
With validation loss: 2.460587501525879


In [26]:
#Load models
if save_load:
    base_path = main_filepath_2 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
tuned_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','NMSE','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=np.linalg.norm(y_test.to_numpy()-preds[:,:,est],axis=(0,1))**2
    norm=np.linalg.norm(y_test.to_numpy(),axis=(0,1))**2
    nmse=mse/norm
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,nmse,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','NMSE','Mean_Quantile_Loss'])
    tuned_model_metrics=pd.concat([tuned_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=np.linalg.norm(y_test.to_numpy()-preds_total[:,:,est],axis=(0,1))**2
norm=np.linalg.norm(y_test.to_numpy(),axis=(0,1))**2
nmse=mse/norm
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,nmse,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','NMSE','Mean_Quantile_Loss'])
tuned_model_metrics=pd.concat([tuned_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    tuned_model_metrics=tuned_model_metrics.reset_index(drop=True)
    tuned_model_metrics.to_csv(main_filepath_2+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_124826/3556501826.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tuned_model_metrics=pd.concat([tuned_model_metrics,tmp_metrics])


Now evaluate:

In [27]:
#Display untuned results
if save_load:
    file_name = main_filepath_1 + '/model_metrics.csv'
    untuned_model_metrics=pd.read_csv(file_name)

untuned_model_metrics

,Ensemble_mean,Ensemble_index,NMSE,Mean_Quantile_Loss
0,False,0.0,0.012683,0.435310
1,False,1.0,0.012534,0.433095
2,False,2.0,0.012403,0.432571
3,False,3.0,0.012364,0.432086
4,False,4.0,0.012604,0.433685
5,True,NaN,0.011956,0.420214


In [28]:
#Display tuned results
if save_load:
    file_name = main_filepath_2 + '/model_metrics.csv'
    tuned_model_metrics=pd.read_csv(file_name)

tuned_model_metrics

,Ensemble_mean,Ensemble_index,NMSE,Mean_Quantile_Loss
0,False,0.0,0.012184,0.424906
1,False,1.0,0.012223,0.425064
2,False,2.0,0.012172,0.423075
3,False,3.0,0.012263,0.424701
4,False,4.0,0.012187,0.424302
5,True,NaN,0.011862,0.416195
